**Imports**

In [1]:
import numpy as np
from numpy.linalg import inv, norm
from math import ceil, log
from linear_solvers import NumPyLinearSolver, HHL
from qiskit import Aer
from qiskit.quantum_info import Statevector

**Setup of HHL helper functions**

**Function for converting an $M\times N$ matrix to $N\times N$**

In [2]:
def make_nxn(matrix, vector):
    if matrix.shape[0] != matrix.shape[1]:
        # set n to the highest value between the rows and columns of matrix
        # by doing this, we can ensure that the matrix will be nxn and the vector will be nx1
        n = max(matrix.shape[0], matrix.shape[1])
        matrix = np.pad(matrix, ((0, n-matrix.shape[0]), (0, n-matrix.shape[1])), 'constant', constant_values = (0))
        vector.resize(1, n, refcheck=False)
        # create a matrix (lambda) that contains a small value to add to the matrix and vector
        # this will ensure that the matrix is invertible and still close in value to the original
        la = np.empty(n)
        la.fill(10**-6)
        la_mat = np.diag(la)
        matrix = matrix + la_mat
        vector = vector + la
    return matrix, vector

**Function for converting an $N\times N$ matrix to a $2^n\times 2^n$ matrix**

In [3]:
def make_2nx2n(matrix, vector):
    # this function assumes that the matrix is square
    # if matrix is not 2^n x 2^n, then set n to the next highest power of 2
    if not log(matrix.shape[0], 2).is_integer():
        n = 2**(ceil(log(matrix.shape[0], 2)))
        matrix = np.pad(matrix, ((0, n-matrix.shape[0]), (0, n-matrix.shape[1])), 'constant', constant_values = (0))
        vector.resize(1, n, refcheck=False)
        for i in range(n):
            if matrix[i][i] == 0:
                matrix[i][i] = 1
    return matrix, vector

**Function for converting an $N\times N$ matrix to a Hermitian matrix**

In [4]:
def make_hermitian(matrix, vector):
    # this function assumes that the matrix is square
    # if the matrix is not hermitian, make it hermitian and alter the vector accordingly
    matrix_H = np.matrix(matrix).H
    if not np.array_equal(matrix, matrix_H):
        zeros = np.zeros((matrix.shape[0], matrix.shape[0]))
        matrix = np.block([[zeros, matrix_H],
                       [matrix, zeros]])
        vector_conj = np.conj(vector)
        vector = np.append(vector_conj, vector)
    return matrix, vector

**Function for making $A$ and $b$ from an equation $Ax=b$ compatible with the HHL algorithm**

In [5]:
def hhl_compatible(matrix, vector):
    mat = matrix.copy()
    vec = vector.copy()
    mat, vec = make_nxn(mat,vec)
    mat, vec = make_2nx2n(mat, vec)
    mat, vec = make_hermitian(mat, vec)
    return mat, vec

**Function for creating HHL object**

In [6]:
def new_HHL():
    backend = Aer.get_backend('aer_simulator')
    hhl = HHL(1e-3, quantum_instance=backend)
    return hhl

**Function for running HHL and extracting required data**

In [7]:
def run_HHL(hhl, matrix, vector):
    mat, vec = hhl_compatible(matrix, vector)
    solution = hhl.solve(mat, vec)
    sv = Statevector(solution.state)
    data_start = int(sv.data.size / 2)
    data_end = data_start + vec.size
    sv_data = sv.data[data_start:data_end].real
    solution_norm = solution.euclidean_norm
    data = solution_norm * sv_data / norm(sv_data)
    vec_norm = norm(vec)
    final_data = (vec_norm * data)[0:vector.size]
    return final_data

**Input Data**

In [8]:
# Bus data
# For each bus the data is as follows:
# 0:BusNum, 1:PG, 2:QG, 3:PD, 4:QD, 5:V, 6:PA, 7:BusType
# Where BusType 1 is Slack, 2 is PV, and 3 is PQ
bus_arr = np.array([[1, 0, 0, -0.9, -0.4, 1.02, 0, 1], 
                  [2, 1.4, 0, 0, 0, 1.03, 0, 2], 
                  [3, 0, 0, -1.1, -0.4, 0, 0, 3]])

# Reactive power limits
reactive_min = []
reactive_max = []
for i in bus_arr:
    if i[7] == 2:
        reactive_min.append(0)
        reactive_max.append(0.8)
    else:
        reactive_min.append(0)
        reactive_max.append(0.8)
reactive_min = np.array(reactive_min)
reactive_max = np.array(reactive_max)

# Branch data
# For each branch the data is as follows:
# 0:BranchBuses, 1:LineImpedance
branch_arr = [[[1, 2], complex(0.025, 0.1)],
             [[1, 3], complex(0.025, 0.1)],
             [[2, 3], complex(0.025, 0.1)]]

**Function for creating Generator and Line Admittance arrays**

In [9]:
def create_GA_LA(Buses, Branches):
    # Impedance Values
    # Line Impedance Matrix
    LI = np.zeros((Buses.shape[0], Buses.shape[0]), dtype = complex)
    for b in Branches:
        # Branch b[0] contains a list of BusNums (BusNums are 1 higher than the index they are at)
        i = b[0][0]-1
        j = b[0][1]-1
        # Set LI for both buses to each other
        LI[i][j] = b[1]
        LI[j][i] = b[1]

    # Compute Generator and Line Admittance
    GA = np.zeros(Buses.shape[0])
    LA = np.divide(1, LI, where = LI!=0)

    return GA, LA

**Function for creating Y<sub>bus</sub>, G, and B**

In [10]:
def create_Ybus(Buses, GA, LA):
    Y = np.zeros((Buses.shape[0], Buses.shape[0]),  dtype = complex)
    for i in range(Y.shape[0]):
        Y[i][i] = Y[i][i] + GA[i]
        for j in range (Y.shape[1]):
            if i != j:
                Y[i, j] = Y[i, j] - LA[i][j]
                Y[i, i] = Y[i, i] + LA[i][j]
                
    G = Y.real
    B = Y.imag

    return Y, G, B

**Function for setting initial values of unknown quantities**

In [11]:
def init_unknown(V, PA):
    # Flat start for bus voltages
    # Unknown values for bus volage will be set to 1
    for i in range(V.size):
        if V[i] == 0:
            V[i] = 1

**Function for computing powers**

In [12]:
def compute_powers(Buses, V, PA, G, B):
    P_new = np.zeros(Buses.shape[0])
    Q_new = np.zeros(Buses.shape[0])
    # We don't need to calculate for the slack bus (the first bus), so we start at the second bus
    for i in range(1, Buses.shape[0]):
        for j in range (Buses.shape[0]):
            P_new[i] = P_new[i]+V[i]*V[j]*(G[i][j]*np.cos(PA[i]-PA[j])+B[i][j]*np.sin(PA[i]-PA[j]))
            Q_new[i] = Q_new[i]+V[i]*V[j]*(G[i][j]*np.sin(PA[i]-PA[j])-B[i][j]*np.cos(PA[i]-PA[j]))

    return P_new, Q_new

**Function for checking reactive power limits**

In [13]:
def check_limits(Buses, V, Q_new, Q_min, Q_max):
    for i in range(1, Buses.shape[0]):
        # Buses[i][7] is bus i's BusType, 2 is PV
        if Buses[i][7] == 2:
            # Check that Q for bus i is within limits
            if (Q_new[i] > Q_max[i]) or (Q_new[i] < Q_min[i]):
                # BusType 3 is PQ
                Buses[i][7] = 3
                # Set Bus's V to initial value
                Buses[i][5] = 1
                V[i] = 1
                return False
    # If the for loop is able to complete, all limit checks pass
    return True

**Function for computing power mismatches**

In [14]:
def compute_power_mismatches(Buses, P, Q, P_new, Q_new):
    Delta_P = []
    Delta_Q = []
    for i in range(1, Buses.shape[0]):
        Delta_P.append(P[i] - P_new[i])
        # Buses[i][7] is Bus i's BusType, 3 is PQ
        if Buses[i][7] == 3:
            Delta_Q.append(Q[i] - Q_new[i])
            
    
    return Delta_P, Delta_Q

**Function for computing the elements of the Jacobian matrix**

In [15]:
def compute_Jacobian(Buses, P_new, Q_new, V, PA, G, B):
    H = np.zeros((Buses.shape[0]-1, Buses.shape[0]-1))
    N = np.zeros((Buses.shape[0]-1, Buses.shape[0]-1))
    J = np.zeros((Buses.shape[0]-1, Buses.shape[0]-1))
    L = np.zeros((Buses.shape[0]-1, Buses.shape[0]-1))
    for i in range(1, Buses.shape[0]):
        for j in range(1, Buses.shape[0]):
            if i == j:
                H[i-1][j-1] = -Q_new[i]-B[i][i]*(V[i]**2)
                # Buses[i][7] is Bus i's BusType, 2 is PV
                if Buses[i][7] != 2:
                    N[i-1][j-1] = P_new[i]+G[i][i]*(V[i]**2)
                    J[i-1][j-1] = P_new[i]-G[i][i]*(V[i]**2)
                    L[i-1][j-1] = Q_new[i]-B[i][i]*(V[i]**2)
            else:
                H[i-1][j-1] = V[i]*V[j]*(G[i][j]*np.cos(PA[i]-PA[j])+B[i][j]*np.sin(PA[i]-PA[j]))
                if Buses[j][7] != 2:
                    N[i-1][j-1] = V[j]*V[i]*(G[i][j]*np.sin(PA[i]-PA[j])+B[i][j]*np.cos(PA[i]-PA[j]))
                if Buses[i][7] != 2:
                    J[i-1][j-1] = -V[i]*V[j]*(G[i][j]*np.cos(PA[i]-PA[j])+B[i][j]*np.sin(PA[i]-PA[j]))
                if Buses[i][7] != 2 and Buses[j][7] != 2:
                    L[i-1][j-1] = V[j]*V[i]*(G[i][j]*np.sin(PA[i]-PA[j])-B[i][j]*np.cos(PA[i]-PA[j]))

    for i in reversed(range(1, Buses.shape[0])):
        if Buses[i][7] == 2:
            # If Bus i is PV, remove column for that bus from N
            N = np.delete(N, i-1, 1)
            # If Bus i is PV, remove row for that bus from J
            J = np.delete(J, i-1, 0)
            # If Bus i is PV, remove row and column for that bus from L
            L = np.delete(L, i-1, 1)
            L = np.delete(L, i-1, 0)

    Jac = np.block([[H, N], [J, L]])
    return Jac

**Function for computing the increment matrix**

In [16]:
def compute_increment(Jac, Delta_P, Delta_Q):
    Delta_PQ = np.append(Delta_P, Delta_Q)
    Delta_PAV = np.dot(inv(Jac), Delta_PQ)
    Delta_PA = Delta_PAV[0:len(Delta_P)]
    Delta_V = Delta_PAV[len(Delta_P):]

    return Delta_PA, Delta_V

**Function for computing the increment matrix (with HHL)**

In [17]:
def compute_increment_hhl(Jac, Delta_P, Delta_Q, hhl):
    Delta_PQ = np.append(Delta_P, Delta_Q)
    Delta_PAV = run_HHL(hhl, Jac, Delta_PQ)
    Delta_PA = Delta_PAV[0:len(Delta_P)]
    Delta_V = Delta_PAV[len(Delta_P):]

    return Delta_PA, Delta_V

**Function for computing new values of $\delta$ and $V$**

In [18]:
def compute_new_PAV(Buses, V, PA, Delta_PA, Delta_V):
    PA_new = np.zeros(Buses.shape[0])
    V_new = np.zeros(Buses.shape[0])
    PA_new[0] = PA[0]
    V_new[0] = V[0]
    incr = 0
    for i in range(1, Buses.shape[0]):
        PA_new[i] = PA[i] + Delta_PA[i-1]
        # Buses[i][7] is Bus i's BusType, 3 is PQ
        if Buses[i][7] == 3:
            V_new[i] = V[i] + (Delta_V[incr])*V[i]
            incr = incr + 1
        else:
            V_new[i] = V[i]
    
    return PA_new, V_new

**Function for checking if $\delta$ and $V$ have converged**

In [19]:
def check_convergence(Buses, V, PA, V_new, PA_new, ep):
    convergence_PA = np.zeros(Buses.shape[0])
    convergence_V = np.zeros(Buses.shape[0])
    for i in range(Buses.shape[0]):
        convergence_PA[i] = abs(PA_new[i] - PA[i])
        convergence_V[i] = abs(V_new[i] - V[i])
    for i in range(Buses.shape[0]):
        if (convergence_PA[i] > ep):
            return False
        # Buses[i][7] is Bus i's BusType, 3 is PQ
        if Buses[i][7] == 3:
            if(convergence_V[i] > ep):
                return False
    # If this line is reached, all convergence levels are <= ep
    return True

**Function for loading calculated values into Bus array**

In [20]:
def load_values(Buses, oldBuses, V_new, PA_new):
    # Load calculated values into Buses and change altered PV buses back to PV
        for i in range(Buses.shape[0]):
            # Buses[i][6] is Bus i's PA
            Buses[i][6] = PA_new[i]
            # Buses[i][5] is Bus i's V
            Buses[i][5] = V_new[i]
            # Buses[i][7] is Bus i's BusType, 3 is PQ, 2 is PV
            if (Buses[i][7] == 3) and (oldBuses[i][7] == 2):
                Buses[i][7] = 2

**Function for calculating injected P and Q values**

In [21]:
def calculate_injected(Buses, G, B):
    for i in range(Buses.shape[0]):
        for j in range(Buses.shape[0]):
            # Buses[i][7] is Bus i's BusType, 3 is PQ
            if Buses[i][7] != 3:
                # Buses[i][2] is Bus i's QG, [i][5] is V, [i][6] is PA
                Buses[i][2] = Buses[i][2]+Buses[i][5]*Buses[j][5]*(G[i][j]*np.sin(Buses[i][6]-Buses[j][6])-B[i][j]*np.cos(Buses[i][6]-Buses[j][6]))
                # 1 is Slack
                if Buses[i][7] == 1:
                    # Buses[i][1] is Bus i's PG
                    Buses[i][1] = Buses[i][1]+Buses[i][5]*Buses[j][5]*(G[i][j]*np.cos(Buses[i][6]-Buses[j][6])+B[i][j]*np.sin(Buses[i][6]-Buses[j][6]))

**Function for using Newton Raphson for Power Flow Problem**

In [22]:
def NR(oldBuses, Branches, Q_min, Q_max, ep):
    # Make a copy of the old buses to retain its data
    Buses = np.copy(oldBuses)

    # Load information on P, Q, V, and PA into variables
    P = []
    Q = []
    V = []
    PA = []
    for i in Buses:
        V.append(i[5])
        PA.append(i[6])
        if i[7] == 2:
            P.append(i[1])
            Q.append(i[2])
        else:
            P.append(i[3])
            Q.append(i[4])
    P = np.array(P)
    Q = np.array(Q)
    V = np.array(V)
    PA = np.array(PA)

    # Value calculations
    GA, LA = create_GA_LA(Buses, Branches)
    Y, G, B = create_Ybus(Buses, GA, LA)
    init_unknown(V, PA)

    total_iterations = 0 # Total iterations of NR
    iterations = 0 # Iterations since limit check last failed
    # Perform NR method iteratively
    # Stop condition handled within the loop
    while(True):
        P_new, Q_new = compute_powers(Buses, V, PA, G, B)
        # If limit check fails, treat ith PV bus as PQ bus and recompute powers
        if not check_limits(Buses, V, Q_new, Q_min, Q_max):
            iterations = 0
            P_new, Q_new = compute_powers(Buses, V, PA, G, B)
        Delta_P, Delta_Q = compute_power_mismatches(Buses, P, Q, P_new, Q_new)
        Jac = compute_Jacobian(Buses, P_new, Q_new, V, PA, G, B)
        Delta_PA, Delta_V = compute_increment(Jac, Delta_P, Delta_Q)
        PA_new, V_new = compute_new_PAV(Buses, V, PA, Delta_PA, Delta_V)
        total_iterations += 1
        iterations += 1
        # If convergence has been reached, exit loop
        if check_convergence(Buses, V, PA, V_new, PA_new, ep):
            break
        V = V_new
        PA = PA_new

    load_values(Buses, oldBuses, V_new, PA_new)    
    calculate_injected(Buses, G, B)

    # Print Data
    print("\nTotal nmber of iterations: ", total_iterations)
    print("Iterations since last limit check failure: ", iterations)
    print("\nOld Voltages:\n", oldBuses[:,5])
    print("\nNew Voltages:\n", Buses[:,5])    
    print("\nOld Phase Angles:\n", oldBuses[:,6])
    print("\nNew Phase Angles:\n", Buses[:,6])
    print("\nOld Bus Values:\n", oldBuses, "\n")
    print("New Bus Values (after NR):\n", Buses, "\n")

    return Buses

**Function for using Newton Raphson for Power Flow Problem (with HHL)**

In [23]:
def NR_hhl(oldBuses, Branches, Q_min, Q_max, ep):
    # Make a copy of the old buses to retain its data
    Buses = np.copy(oldBuses)

    # Load information on P, Q, V, and PA into variables
    P = []
    Q = []
    V = []
    PA = []
    for i in Buses:
        V.append(i[5])
        PA.append(i[6])
        if i[7] == 2:
            P.append(i[1])
            Q.append(i[2])
        else:
            P.append(i[3])
            Q.append(i[4])
    P = np.array(P)
    Q = np.array(Q)
    V = np.array(V)
    PA = np.array(PA)

    # Value calculations
    GA, LA = create_GA_LA(Buses, Branches)
    Y, G, B = create_Ybus(Buses, GA, LA)
    init_unknown(V, PA)

    total_iterations = 0 # Total iterations of NR
    iterations = 0 # Iterations since limit check last failed

    hhl = new_HHL() # HHL setup for running compute_increment_hhl()

    # Perform NR method iteratively
    # Stop condition handled within the loop
    while(True):
        P_new, Q_new = compute_powers(Buses, V, PA, G, B)
        # If limit check fails, treat ith PV bus as PQ bus and recompute powers
        if not check_limits(Buses, V, Q_new, Q_min, Q_max):
            iterations = 0
            P_new, Q_new = compute_powers(Buses, V, PA, G, B)
        Delta_P, Delta_Q = compute_power_mismatches(Buses, P, Q, P_new, Q_new)
        Jac = compute_Jacobian(Buses, P_new, Q_new, V, PA, G, B)
        Delta_PA, Delta_V = compute_increment_hhl(Jac, Delta_P, Delta_Q, hhl)
        Delta_PA2, Delta_V2 = compute_increment(Jac, Delta_P, Delta_Q)
        PA_new, V_new = compute_new_PAV(Buses, V, PA, Delta_PA, Delta_V)
        total_iterations += 1
        iterations += 1

        # Compare HHL with Classical Method
        print("Delta_PA difference at iteration ", total_iterations, ":\n", abs(Delta_PA2 - Delta_PA))
        print("Delta_V difference at iteration ", total_iterations, ":\n", abs(Delta_V2 - Delta_V))

        # If convergence has been reached, exit loop
        if check_convergence(Buses, V, PA, V_new, PA_new, ep):
            break
        V = V_new
        PA = PA_new

    load_values(Buses, oldBuses, V_new, PA_new)    
    calculate_injected(Buses, G, B)

    # Print Data
    print("\nTotal nmber of iterations: ", total_iterations)
    print("Iterations since last limit check failure: ", iterations)
    print("\nOld Voltages:\n", oldBuses[:,5])
    print("\nNew Voltages:\n", Buses[:,5])    
    print("\nOld Phase Angles:\n", oldBuses[:,6])
    print("\nNew Phase Angles:\n", Buses[:,6])
    print("\nOld Bus Values:\n", oldBuses, "\n")
    print("New Bus Values (after NR):\n", Buses, "\n")

    return Buses

In [31]:
bus = NR(bus_arr, branch_arr, reactive_min, reactive_max, 10**-2)


Total nmber of iterations:  4
Iterations since last limit check failure:  4

Old Voltages:
 [1.02 1.03 0.  ]

New Voltages:
 [1.02       1.03       0.98984173]

Old Phase Angles:
 [0. 0. 0.]

New Phase Angles:
 [ 0.          0.05198922 -0.02153548]

Old Bus Values:
 [[ 1.    0.    0.   -0.9  -0.4   1.02  0.    1.  ]
 [ 2.    1.4   0.    0.    0.    1.03  0.    2.  ]
 [ 3.    0.    0.   -1.1  -0.4   0.    0.    3.  ]] 

New Bus Values (after NR):
 [[ 1.         -0.25694251  0.28638613 -0.9        -0.4         1.02
   0.          1.        ]
 [ 2.          1.4         0.22084515  0.          0.          1.03
   0.05198922  2.        ]
 [ 3.          0.          0.         -1.1        -0.4         0.98984173
  -0.02153548  3.        ]] 



In [32]:
bus_hhl = NR_hhl(bus_arr, branch_arr, reactive_min, reactive_max, 10**-2)

Delta_PA difference at iteration  1 :
 [0.00022946 0.00045728]
Delta_V difference at iteration  1 :
 [4.05508227e-05]
Delta_PA difference at iteration  2 :
 [0.00013829 0.0002577 ]
Delta_V difference at iteration  2 :
 [0.00040551]
Delta_PA difference at iteration  3 :
 [1.86096038e-05 3.83762270e-05]
Delta_V difference at iteration  3 :
 [0.00011536]
Delta_PA difference at iteration  4 :
 [8.92142971e-06 5.93295141e-05]
Delta_V difference at iteration  4 :
 [4.01765301e-05]

Total nmber of iterations:  4
Iterations since last limit check failure:  4

Old Voltages:
 [1.02 1.03 0.  ]

New Voltages:
 [1.02       1.03       0.98989312]

Old Phase Angles:
 [0. 0. 0.]

New Phase Angles:
 [ 0.          0.05200356 -0.02154434]

Old Bus Values:
 [[ 1.    0.    0.   -0.9  -0.4   1.02  0.    1.  ]
 [ 2.    1.4   0.    0.    0.    1.03  0.    2.  ]
 [ 3.    0.    0.   -1.1  -0.4   0.    0.    3.  ]] 

New Bus Values (after NR):
 [[ 1.         -0.25711034  0.28591379 -0.9        -0.4         1.02
